# Importing, need beautifulsoup urllib(v3) requests and lxml's html

In [1]:
from bs4 import BeautifulSoup
import urllib3
import requests
from lxml import html

### get_website() makes a call to the website, and instantiates a beautiful soup object

In [2]:
def get_website():
    url = 'http://rosettacode.org/wiki/Copy_a_string'
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'lxml')
    return soup

### get_links() searches the html on the website and returns objects from the websites code

In [ ]:
def get_links():
    prefix = 'http://rosettacode.org'
    find_link = soup.select('td')
    return [prefix + line.a["href"] for line in find_link]

### get_title() uses the beautiful soup object from get_website() and adds anything formated in header2 to the list titles

In [ ]:
def get_title(soup):
    titles = []
    posts = soup.select("h2 > span")
    for post in posts:
        if post.text != "[edit]":
            titles.append(post.text)
    return titles

### get_code_examples() takes the website and any titles and creates a dictionary with titles as the key and anything found in the html class for that title as the value

In [3]:
def get_code_examples(soup, languages):
    language_list = dict((x, []) for x in languages)
    lines = soup.select('pre')
    for line in lines:
        if line.has_attr("class"):
            for key, value in language_list.items():
                if line['class'][0] == key.lower():
                    language_list[key].append(line.text)
    return language_list

### when it's all put together it looks something like this and I've thrown it all into a dataframe to be processed by a clasifyer

In [4]:
soupy = get_website()

In [5]:
langy = get_title(soupy)

In [6]:
dic = get_code_examples(soupy,langy)

In [7]:
df = pd.DataFrame(list(dic.items()))

In [8]:
df.head()

,0,1
0,Shiny,[]
1,Neko,[]
2,Clojure,"[(let [s ""hello"" s1 s] (println s s1))]"
3,AWK,"[BEGIN { a = ""a string"" b = a sub(/a/, ""X"",..."
4,NetRexx,[/* NetRexx */options replace format comments ...


## this is a classifyer that I need to figure out before I can report on findings

In [10]:
import math

class LangClassifier():
    """
    Inspired by http://stackoverflow.com/questions/475033/detecting-programming-language-from-a-snippet
    """
    def __init__(self):
        self.data = {}
        self.totals = {}

    def words(self, code):
        word_list = code.split()
        return filter(bool, word_list)

    def train(self, code, lang):
        # Trains the classifier
        self.data[lang] = {}
        for word in self.words(code):
            if word in self.data[lang]:
                self.data[lang][word] += 1
            else:
                self.data[lang][word] = 1
            if word in self.totals:
                self.totals[word] += 1
            else:
                self.totals[word] = 1
    
    def prob(self, words, lang):
        # Calculates the probability
        res = 0.0
        for word in words:
            try:
                res = res + math.log(self.totals[word]/self.data[lang][word])
            except(KeyError):
                continue
        return res
    
    def classify(self, code):
        # Classifies the input code
        lang_prob = {}
        words = self.words(code)
        for lang in self.data.iterkeys():
            prob = self.prob(words, lang)
            lang_prob[prob] = lang
        return "Input file is most likely: " + lang_prob[min(lang_prob.keys())]

In [18]:
LangClassifier(df)

for row in df:
    LangClassifier.classify(LangClassifier, dic)

TypeError: __init__() takes 1 positional argument but 2 were given